installiong required libraries

In [ ]:
%pip install intel-extension-for-pytorch

In [ ]:
%pip install sentence-transformers

In [ ]:
%pip install intel-extension-for-pytorch

In [ ]:
import torch
import intel_extension_for_pytorch as ipex

loading models from sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

model_embed = SentenceTransformer("BAAI/bge-m3").eval()

We have put the model on eval model because Intel extension supports this method to load on Intel architecture

we're using ipex to optimize the embedding model for intel architecture (CPU only)

In [ ]:
model_embed = ipex.optimize(model_embed)

In [ ]:
query_embedding = model_embed.encode('How big is London')
passage_embedding = model_embed.encode(['London has 9,787,426 inhabitants at the 2011 census', 'London is known for its finacial district'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Now, loading any random HF model of your choice and let's get started with Intel hardware

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "gpt2"

model = AutoModelForCausalLM.from_pretrained(model_path).eval()
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
dtype = torch.float
model = ipex.llm.optimize(model, dtype=dtype)

In [ ]:
input_prompt = "Once upon a time"
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')

In [ ]:
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)

In [ ]:
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Input prompt: {input_prompt}")
print(f"Generated text: {generated_text}")